# A full business solution

## Following `day2.ipynb`, this code adds a gradio interface to W1D5 notebooks so people can do it on the internet!

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company from its primary website to be used for prospective clients, investors and potential recruits.

*And deploy a gradio app*

In [24]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

import gradio as gr

In [25]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [26]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [27]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [28]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [29]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [30]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [31]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/outsmart'}]}

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [32]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
deepseek-ai/DeepSeek-V3-Base
Updated
1 day ago
•
2.33k
•
762
deepseek-ai/DeepSee

In [33]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt_funny = "You are an assistant that analyzes the contents of several relevant pages from a company website \
 and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
 Include details of company culture, customers and careers/jobs if you have the information."


In [34]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [26]:
print(get_brochure_user_prompt("HuggingFace", "https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://discuss.huggingface.co'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
You are looking at a company called: HuggingFace
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datase

In [35]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

def create_brochure_system_prompt_options(company_name, url,system_prompt_opt):
    """" Créé pour l'exercice final """
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_opt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'docs page', 'url': 'https://huggingface.com/docs'}]}


```markdown
# Hugging Face: The AI Community Building the Future

Welcome to **Hugging Face**, a vibrant platform where the machine learning community collaborates to build, share, and enhance models, datasets, and applications. As a pioneering company in AI, we are dedicated to making machine learning accessible for everyone.

---

## Our Mission
At Hugging Face, we are committed to fostering collaboration within the machine learning community. We believe in the collective intelligence of our users, which drives innovation and accelerates advancements in AI technology.

## What We Provide
- **Models**: Browse through over **400,000 models** including recent trending models like *Llama-3.3-70B-Instruct* and *HunyuanVideo*.
- **Datasets**: Access more than **100,000 datasets** geared for various AI tasks including computer vision, audio, and natural language processing.
- **Spaces**: Explore applications with **150,000+ interactive applications** that showcase cutting-edge AI technology.

### Core Features
- **Open Source Resources**: We provide a vast repository of tools and libraries including Transformers, Diffusers, and Tokenizers, empowering developers to create state-of-the-art machine learning solutions.
- **Enterprise Solutions**: Customized plans with enterprise-grade security for organizations to deploy AI models efficiently.
- **Community Engagement**: Join a thriving community of over **50,000 organizations** including tech giants like Meta, Amazon, Google, and Microsoft. Engage in collaboration, share knowledge, and enhance your AI skills with like-minded professionals.

---

## Company Culture
At Hugging Face, we embrace a culture of openness, collaboration, and continuous learning. Our community-driven approach fosters innovation and encourages our team members to explore new ideas and solutions. We are deeply committed to building a diverse and inclusive environment that represents all voices within the AI landscape.

---

## Careers at Hugging Face
Join us in shaping the future of machine learning! We are always looking for talented individuals who are passionate about AI, technology, and collaboration. If you are interested in a career that makes a difference, check out our current openings on our [Jobs page](https://huggingface.co/jobs).

### Current Job Openings
- Data Scientist
- Machine Learning Engineer
- Community Advocate
- Product Manager

---

## Connect with Us
Experience the power of collaborative AI. 
- **Follow us on Social Media**: [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface) | [GitHub](https://github.com/huggingface)
- **Join our Forum**: Engage with us on [Discord](https://huggingface.co/discord) and share your insights and projects.

---

**Hugging Face**: Where the future of AI is built, one collaboration at a time.
```

In [24]:
create_brochure("HAS", "https://www.has-sante.fr/")
# create_brochure("BCG","https://www.bcg.com/france")


Found links: {'links': [{'type': 'home page', 'url': 'https://www.has-sante.fr'}, {'type': 'about page', 'url': 'https://www.has-sante.fr/jcms/fc_2874838/fr/la-has'}, {'type': 'careers page', 'url': 'https://www.has-sante.fr/jcms/p_3107968/fr/nous-rejoindre'}, {'type': 'press page', 'url': 'https://www.has-sante.fr/jcms/fc_2874839/fr/presse'}, {'type': 'missions page', 'url': 'https://www.has-sante.fr/jcms/c_1002212/fr/missions-de-la-has'}, {'type': 'publications page', 'url': 'https://www.has-sante.fr/jcms/prd1_2989647/fr/toutes-nos-publications'}]}


# Brochure: Haute Autorité de Santé (HAS)

---

## About HAS

The Haute Autorité de Santé (HAS) is an independent public authority in France dedicated to improving the quality of healthcare. Our mission is to enhance the safety and quality of health services provided to patients while ensuring coherence in care pathways across the healthcare system.

### Our Services
- **Patient Safety**: We provide guidelines and frameworks to engage healthcare professionals in creating a safer environment for patients.
- **Healthcare Coordination**: Our work includes organizing care pathways and enhancing cooperation among healthcare professionals to ensure comprehensive patient care.
- **Professional Evaluation**: We assess the effectiveness of medical practices and medical devices, as well as oversee the accreditation of healthcare professionals.

---

## Company Culture

At HAS, we foster a culture of excellence, integrity, and collaboration. Our team is committed to making impactful decisions that are guided by scientific evidence. We believe in transparency and communication, strengthening our partnerships with various healthcare stakeholders, including professionals, institutions, and industries.

We value continuous learning and innovation, ensuring that our team members have opportunities for professional growth and development. 

---

## Our Customers

Our primary customers include healthcare professionals and institutions, including hospitals and clinics, who rely on our evaluations, recommendations, and training materials. We also engage with manufacturers of medical devices and medicines to ensure that quality standards are upheld. Ultimately, our most important customers are the patients who benefit from our work, as we strive to enhance their access to high-quality healthcare services.

---

## Careers at HAS

Join us in our mission to improve health care quality and safety! At HAS, we are always looking for dedicated and skilled professionals to join our team. 

### Opportunities Include:
- Healthcare assessors
- Quality and safety coordinators
- Healthcare policy analysts
- Administrative and support roles

**Why Work with Us?**
- Competitive salary and benefits
- Commitment to work-life balance
- Opportunities for professional growth
- A collaborative and inclusive environment

---

## Get In Touch

For more information or to explore career opportunities with HAS, visit our website or contact us directly at:

- **Website**: [hauteautoritedesante.fr](http://hauteautoritedesante.fr)
- **Email**: contact@has.fr

---

Join us at the Haute Autorité de Santé as we work together to innovate and create a safer, more effective healthcare system for everyone!

In [19]:
create_brochure_system_prompt_options("BPI - France","https://www.bpifrance.fr/",system_prompt_funny)

Found links: {'links': [{'type': 'about page', 'url': 'https://www.bpifrance.fr/nous-decouvrir'}, {'type': 'careers page', 'url': 'https://www.bpifrance.fr/nous-decouvrir/rejoignez-nous'}, {'type': 'company page', 'url': 'https://www.bpifrance.fr/nous-decouvrir/notre-mission'}, {'type': 'partners page', 'url': 'https://www.bpifrance.fr/nous-decouvrir/nos-partenaires'}, {'type': 'commitments page', 'url': 'https://www.bpifrance.fr/nos-engagements'}, {'type': 'solutions page', 'url': 'https://www.bpifrance.fr/nos-solutions'}, {'type': 'contact page', 'url': 'https://bpifrance.fr/contactez-nous'}, {'type': 'investors page', 'url': 'https://www.bpifrance.fr/Espace-Investisseurs'}]}


# Welcome to the Wacky World of Bpifrance! 🎉

## Who are we? 
We're Bpifrance, but you can call us "The Bank of Entrepreneurs" - no need to call us a cab because we’re already here to help you drive your ambitious dreams! 🚗💨 Our mission? To serve the future while giving your projects all the support they need, from creation to international expansion. Our motto? "Don't let your dreams be dreams; let us help you make them REAL!" 

## Our Culture: Where Every Day is 'Bring Your Ideas to Work' Day 🌟
At Bpifrance, coffee isn’t the only thing that’s strong; our team is a dynamic group of enthusiastic individuals dedicated to nurturing the entrepreneurial spirit. With over 1,200 fantastic folks scattered across 50 posts in places like Nouvelle-Aquitaine and Auvergne-Rhône-Alpes, we’ve got the French market under our... berets. And if you don’t get a beret, don’t worry! You’ll get plenty of chances to create, innovate, and—who are we kidding?—over-caffeinate. ☕

## Our Customers: The Future Tycoons of the Universe! 🌌
Whether you're a daring start-up or a well-established business, we cater to every entrepreneur—from the one-man band in a basement to massive corporations ready to make a lasting impact! Join the ranks of innovative creators and climate heroes—after all, *Climat cherche patrons militants*! So, if you want a supportive partner who’ll be as dedicated as your mom during family game night, then we are the right choice for you!

## Services Galore: Your Swiss Army Knife for Business Solutions 🛠️
Got projects? We’ve got solutions!
- **Creation**: Got an idea? Let’s turn that idea into a venture that even your grandma will brag about!
- **Innovation**: We’ll throw some funding in your innovation soup until it’s just right. 🍲
- **Financing & Investments**: Need cash? We’re like your friendly neighborhood chef with a secret menu for financing your ambitions.
- **Accompaniment**: Think of us as your personal growth coach, but without the shouting or necessity of Lycra.

## Careers: Join the Coolest Crew in Town! 🚀
Are you ready to leap into a world where innovation meets collaboration? If you’re looking for a place where coffee runs are epic, meetings involve more brainstorming than bunting, and every achievement is celebrated like it’s Bastille Day, Bpifrance has your seat waiting. 

### Open Positions
Whether you're looking for middle management or an opportunity to wear all the hats in a start-up maritime adventure (okay, maybe not the hats), we have roles for everyone!

## Contact Us
Want to climb aboard the Bpifrance train? All aboard! Give us a shout:
- 📧 Email: Don’t worry, we’re super responsive and our email is always open like our policies for entrepreneurs! 
- 📞 Phone: Let us hear your entrepreneurial spirit over the phone. We might even share a laugh or two!

---

**So why choose Bpifrance?** Because we are “La banque du climat” with the best of the private and public sectors combined. Plus, we promise to keep your entrepreneurial ambition as vibrant as our croissants on Sunday mornings.🥐

**Bpifrance: Serving the Future—One Entrepreneur at a Time!**

### using gradio :

In [36]:
input1 = gr.Textbox(label = "Organisation : ")
input2 = gr.Textbox(label = "URL : ")
input3 = gr.Dropdown([system_prompt,system_prompt_funny], label = "Mode :", value = system_prompt_funny)

In [37]:
view = gr.Interface(
    fn=create_brochure_system_prompt_options,
    inputs=[input1, input2, input3],
    outputs=[gr.Markdown(label="Résultat : ")],
    flagging_mode="never"
)
view.launch(share = True)

* Running on local URL:  http://127.0.0.1:7870
* Running on public URL: https://ea5de5631a4156d7a4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Found links: {'links': [{'type': 'about page', 'url': 'https://www.cnc.fr/a-propos-du-cnc'}, {'type': 'careers page', 'url': 'https://www.cnc.fr/a-propos-du-cnc/recrutement'}, {'type': 'contact page', 'url': 'https://www.cnc.fr/contact'}, {'type': 'company page', 'url': 'https://www.cnc.fr/web/en'}, {'type': 'professionals page', 'url': 'https://www.cnc.fr/professionnels'}]}


# Welcome to CNC: The Cinematic Wonderland! 🎬✨

**Are you ready for a journey through the dazzling worlds of cinema, TV, and video games?** Look no further! The CNC (Centre National du Cinéma et de l'image animée) is your one-stop-shop for all things vibrant and visual.

---

## Who Are We?

At CNC, our mission is as clear as the plot of a rom-com: **Support the creation of cinema, series, TV shows, and video games!** From developing groundbreaking scripts to producing mind-bending games and curating fabulous films, we’re in the business of creativity—and we do it well!

---

## What Can You Expect? 😄

### 🎥 Cinema Galore!
- **Scriptwriting Superheroes**: We provide assistance like a good sidekick supports a hero, helping authors fine-tune their genius scripts.
- **Patrimoine Cinematographique**: A fancy way of saying we protect and cherish our cinematic treasures like they're our pet goldfish.

### 📺 TV and Series Shenanigans!
- We're the fairy godmothers of television! Imagine helpful guides and resources that turn dreams into screen time. ⭐
- Programs like "Ma classe au cinéma" for schools, because who said classrooms can't be epic studio sets?

### 🎮 Gaming Awesomeness!
- Got a game idea? We’ll make it go from "meh" to "WOW" faster than you can say "Play it again!" 🎮
- Plus, we help with technical support faster than you can say “What’s the Wi-Fi password?” 

---

## Our Guests 👥

Our patrons range from fresh auteurs thinking big 🎉 to established creators echoing historic dialogues. Whether you’re here to turn your life story into a gripping drama or seeking help for that quirky video game idea where cats rule the world, we’ve got the resources for you!

---

## Our Culture: Work, Play, and Fun! 🎉

At CNC, teamwork makes the dream work! 
- **Diversity and Inclusion**: From directors to artists, everyone’s voice matters—unless you're a mime; then we kindly ask you to help out without speaking.
- **Social Responsibility**: We strive to keep the creative world safe, nurturing, and welcoming. Yes, there's a training for tackling sex-based violence because everyone deserves a safe space to create!
- **Fun & Frolic**: Expect game nights, movie marathons, and playful brainstorming sessions that sparkle with creativity.

---

## Join Us! 🔍

Interested in becoming a part of this cinematic circus? Whether you're a film buff, a gaming guru, or someone with a wild idea for a TV show, we might be looking for you! 
- Our motto? **"If you can dream it, you can film it... or game it!"**

Let’s turn your dreams into scenes!

---

Don't miss your chance! Roll the credits and join the fun! 🌟 

### For Inquiries, Contact Us!
[Visit our website](#) if you’re ready to step into the studio of dreams. 

---

**CNC: Because life is better in HD!** 📼✨

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cnc.fr/a-propos-du-cnc'}, {'type': 'contact page', 'url': 'https://www.cnc.fr/contact'}, {'type': 'careers page', 'url': 'https://www.cnc.fr/a-propos-du-cnc/recrutement'}, {'type': 'professionals page', 'url': 'https://www.cnc.fr/professionnels'}, {'type': 'company page', 'url': 'https://www.cnc.fr/web/en'}]}


# CNC Company Brochure

## About CNC

CNC (Centre National du Cinéma et de l'image animée) plays a vital role in supporting the film, TV series, video games, and digital creation sectors. It provides resources, funding, and expert guidance to creators and professionals to foster innovation and artistic exploration across various media.

### Mission
- **Support for Creation**: Providing financial and advisory support for cinema, television, video games, and digital creation.
- **Education**: Promoting awareness and education in visual storytelling, starting from a young age with programs like "Ma classe au cinéma."
- **Cultural Heritage**: Upholding and promoting France's cinematic heritage while facilitating access to audiovisual works.

## Company Culture

At CNC, the work environment is shaped by a commitment to creativity, inclusion, and innovation. The organization values diversity in its workforce and focuses on creating a safe and supportive space for all employees. CNC is proactive in combating gender-based violence and is dedicated to raising awareness through training programs.

### Corporate Social Responsibility
- Implementation of policies that underline the importance of accessibility to media and spaces.
- Active measures to prevent sexual and gender-based violence.
- Promoting environmental responsibility and sustainability in its operations.

## Customers and Stakeholders

CNC caters to a diverse range of stakeholders, including:
- **Creators**: Offering comprehensive resources for authors, screenwriters, and filmmakers.
- **Students and Educators**: Programs aimed at developing visual literacy and a passion for cinema and storytelling.
- **Industry Professionals**: Providing access to funding and technical support across various mediums, including film, television, and games.

## Careers at CNC

CNC offers various career opportunities for individuals passionate about culture, arts, and media. It seeks talent that is eager to contribute to the evolution of audiovisual arts.

### Opportunities Include:
- **Creative Roles**: Supporting writers, directors, and producers in their projects.
- **Administrative and Support Positions**: Helping streamline processes and provide resources to creators.
- **Research and Analysis**: Conducting studies and statistics to inform policy and program development.

### Join Us!
If you are interested in building a career with a significant impact on creative industries, CNC might be the place for you! 

## Contact Us

For more information about CNC, our services, or career opportunities, please visit our website or contact us directly!

---

CNC invites you to be part of a forward-thinking organization dedicated to nurturing creative talent and advancing the fields of cinema, television, and digital arts.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [13]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
# stream_brochure("HuggingFace", "https://huggingface.co")

In [31]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

create_brochure_system_prompt_options("HAS", "https://www.has-sante.fr/",system_prompt_funny)

Found links: {'links': [{'type': 'home page', 'url': 'https://www.has-sante.fr/'}, {'type': 'about page', 'url': 'https://www.has-sante.fr/jcms/fc_2874838/fr/la-has'}, {'type': 'careers page', 'url': 'https://www.has-sante.fr/jcms/p_3107968/fr/nous-rejoindre'}, {'type': 'press page', 'url': 'https://www.has-sante.fr/jcms/fc_2874839/fr/presse'}, {'type': 'mission page', 'url': 'https://www.has-sante.fr/jcms/c_1002212/fr/missions-de-la-has'}, {'type': 'publications page', 'url': 'https://www.has-sante.fr/jcms/prd1_2989647/fr/toutes-nos-publications'}]}


# Welcome to the Wacky World of HAS!

The Haute Autorité de Santé (HAS)! Not only can we pronounce it, but we also guarantee you’ll never forget it. Here’s why you’ll want to join us on this merry healthcare journey!

### 🎭 **Who Are We?**

At HAS, we don’t just evaluate health standards; we orchestrate the symphony of healthcare. Think of us as the conductors ensuring every note hits perfectly: patient safety, professional evaluations, and the coordination of care are just a few of our many moving parts!

### 💼 **Careers at HAS: Join the Fun!**

- **Doctoral Dynamo:** Whether you’re a medical maestro or a healthcare hero, we want you to lead the charge in improving patient care!
- **Policy Wizards:** Do you dream in regulations? We need your expertise to conjure up standards that keep everyone safe and sound.
- **Data Detectives:** Help us solve the mysteries of healthcare quality as we evaluate, analyze, and recommend how to improve it for all!

### 👩‍⚕️ **Our Customers: Who Do We Serve?**

- **Patients:** They're on the forefront, and we ensure their health and safety is as shiny as a new penny!
- **Healthcare Professionals:** We provide the guidelines and evaluations—think of us as the rule-makers at the healthcare block party!
- **Institutions:** Hospitals, clinics, you name it—we give them the thumbs up to keep providing top-notch care!

### 🤝 **Culture: A Place Like No Other!**

- **Fun Fridays:** Catch a glimpse of our wacky team bonding sessions—just don’t mind the doc in a tutu. 
- **Inclusive Environment:** Everyone is welcome here! Just bring your best ideas and snacks—snack enthusiasts are highly valued!
- **Innovative Spirit:** Got a brilliant idea about improving healthcare? We’re all ears (and probably some exciting healthcare innovation tech too)!

### 📅 **Upcoming Events: Don’t Miss Out!**

- **Colloque HAS/ANM:** "Vaccination et sujets âgés" – because who doesn’t want to talk about health with a side of cake? 
- **Health Innovation Panel:** Join us in looking forward to the future of health—it’s going to be a wild ride!

### 🎉 **Join Us!**

If you’re excited to embark on an enchanting adventure in healthcare, HAS is the place! Together, we’ll change lives and maybe even wear funny hats along the way. 

---

Ready to hop aboard this healthcare rollercoaster? Don’t just stand there—join us and let’s bring some excitement to health standards together! 🎢💙

In [16]:
create_brochure_system_prompt_options("BCG", "https://www.bcg.com/france",system_prompt_funny)

Found links: {'links': []}


# Welcome to BCG: Where Even Our Website Has Secrets!

### Who Are We?
**BCG** (also known as the Busy Consultative Geese) is a top-tier consultancy firm that prides itself on being as enigmatic as it is effective. While our landing page seems to be reinventing the concept of “mystery box,” we assure you, our services are not as exclusive as our web access.

### Culture: Feathers and Fortune
At BCG, we believe in a workplace culture that celebrates growth (and the occasional duck pond break). Our employees enjoy:
- **Collaborative Collaboration:** Yes, we invented this term—anyone can collaborate, but collaborating collaboratively? That's next level!
- **Innovation on the Loose:** Our ideas are like geese in flight, soaring high and occasionally honking at unforeseen challenges.
- **Work-Life Balance:** We encourage you to take breaks...just make sure you exit the browser before reentering your world of productivity!

### Our Clients: The Unflappable Ones
We cater to an array of distinguished clients—from Fortune 500 companies wearing ties to innovative startups that think outside the box (in fact, they may have never even met a box). Our clients rely on us for insights that are as sound as a marching band of geese!

### Careers: Fly High with Us!
Looking for a job? Join our flock! Here are some reasons why you should:
- **Daring Duos:** Partner with fellow masterminds to tackle the toughest business questions. More exciting than a crossword puzzle!
- **Training This Way Comes:** Get ready for training sessions that might involve fewer goose puns and more intellect-boosting shenanigans.
- **Plenty of Opportunities:** As long as you can access the website, there’s a career path for every brave soul willing to jump into the consultancy lake.

### Warning: Enter at Your Own Risk!
We’ve locked our homepage behind a gate, but fear not! Our consultants and quirky company culture are waiting for you to discover beyond the access denied. Whether you’re a customer looking for strategic consulting services, an investor seeking the next big thing, or a potential recruit ready to spread your wings—we promise there’s more than just breadcrumbs waiting for you!

### Join the BCG Flock!
Don’t miss your chance to be part of a team that can turn business problems into solutions, one feather at a time. We might deny you access for now, but that just means you'll have to dive deeper to win our hearts.

Ready to take off with BCG? We hope so, because our doors (once accessed properly) are wide open for you!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>